In [1]:
%load_ext autoreload
%autoreload 2

from kedro.io import AbstractVersionedDataset
import pandas as pd
import rich.pretty
from iceberg_incremental.datasets import PyIcebergDataset
rich.pretty.install()

In [2]:
!curl https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet -o /tmp/yellow_tripdata_2023-01.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0  6196k      0  0:00:07  0:00:07 --:--:-- 9352k


# First create a table 

In [5]:
# Downcast timestamp for compatability https://github.com/apache/iceberg-python/issues/1045#issuecomment-2445205707
import os
os.environ['PYICEBERG_DOWNCAST_NS_TIMESTAMP_TO_US_ON_WRITE'] = 'true'

In [8]:
dataset = PyIcebergDataset("taxi_dataset")
df = pd.read_parquet("/tmp/yellow_tripdata_2023-01.parquet")

In [9]:
dataset.save(df)

Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/table/__init__.py:558: UserWarning: Delete operation did not match any records
  warnings.warn("Delete operation did not match any records")
Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.name() function instead.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in

If the table does not exists, the dataset will create an Iceberg table automatically. One way to verify this is using the `pyiceberg` CLI.

The uri is hard coded as `sqlite:///tmp/warehouse/pyiceberg_catalog.db". Normally you would not do this but this make the Iceberg table transparent within the directory for inspection.

In [10]:
uri = "sqlite:///tmp/warehouse/pyiceberg_catalog.db"

## Using PyIceberg CLI
```bash
➜  pyiceberg --help
Usage: pyiceberg [OPTIONS] COMMAND [ARGS]...

Options:
--catalog TEXT
--verbose BOOLEAN
--output [text|json]
--ugi TEXT
--uri TEXT
--credential TEXT
--help                Show this message and exit.

Commands:
describe    Describes a namespace xor table
drop        Operations to drop a namespace or table
list        Lists tables or namespaces
location    Returns the location of the table
properties  Properties on tables/namespaces
rename      Renames a table
schema      Gets the schema of the table
spec        Returns the partition spec of the table
uuid        Returns the UUID of the table
```

In [ ]:
# List namespace
!pyiceberg --uri $uri list

/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:35: DeprecationWarning: Call to DEFAULT_MIN_SNAPSHOTS_TO_KEEP, deprecated in 0.8.0, will be removed in 0.9.0. DEFAULT_MIN_SNAPSHOTS_TO_KEEP is deprecated. Use TableProperties.MIN_SNAPSHOTS_TO_KEEP_DEFAULT instead..
  _deprecation_warning(message)
/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:35: DeprecationWarning: Call to DEFAULT_MAX_SNAPSHOT_AGE_MS, deprecated in 0.8.0, will be removed in 0.9.0. DEFAULT_MAX_SNAPSHOT_AGE_MS is deprecated. Use TableProperties.MAX_SNAPSHOT_AGE_MS_DEFAULT instead..
  _deprecation_warning(message)
default


In [ ]:
# list table inside default namespace
!pyiceberg --uri $uri list default

/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:35: DeprecationWarning: Call to DEFAULT_MIN_SNAPSHOTS_TO_KEEP, deprecated in 0.8.0, will be removed in 0.9.0. DEFAULT_MIN_SNAPSHOTS_TO_KEEP is deprecated. Use TableProperties.MIN_SNAPSHOTS_TO_KEEP_DEFAULT instead..
  _deprecation_warning(message)
/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:35: DeprecationWarning: Call to DEFAULT_MAX_SNAPSHOT_AGE_MS, deprecated in 0.8.0, will be removed in 0.9.0. DEFAULT_MAX_SNAPSHOT_AGE_MS is deprecated. Use TableProperties.MAX_SNAPSHOT_AGE_MS_DEFAULT instead..
  _deprecation_warning(message)
default.taxi_dataset


In [15]:
!pyiceberg --uri $uri describe default.taxi_dataset

/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:35: DeprecationWarning: Call to DEFAULT_MIN_SNAPSHOTS_TO_KEEP, deprecated in 0.8.0, will be removed in 0.9.0. DEFAULT_MIN_SNAPSHOTS_TO_KEEP is deprecated. Use TableProperties.MIN_SNAPSHOTS_TO_KEEP_DEFAULT instead..
  _deprecation_warning(message)
/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:35: DeprecationWarning: Call to DEFAULT_MAX_SNAPSHOT_AGE_MS, deprecated in 0.8.0, will be removed in 0.9.0. DEFAULT_MAX_SNAPSHOT_AGE_MS is deprecated. Use TableProperties.MAX_SNAPSHOT_AGE_MS_DEFAULT instead..
  _deprecation_warning(message)
Table format version  2                                                         
Metadata location     file://tmp/warehouse/default.db/taxi_dataset/metadata/000…
Table UUID            7c224d55-a9ee-4e16-983b-ee3d33b54e19                      
Last Updated          1732274297944                                

## Load Data

In [16]:
data = dataset.load()

In [17]:
data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Overwrite data

In [18]:
dataset._table.last_sequence_number

1

In [19]:
dataset.save(data)

Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
Iceberg does not yet support 'ns' timestamp precision. Downcasting to 'us'.
/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Table.identifier property is deprecated. Please use Table.name() function instead.
  _deprecation_warning(deprecation_notice(deprecated_in, removed_in, help_message))
/Users/Nok_Lam_Chan/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/utils/deprecated.py:54: DeprecationWarning: Deprecated in 0.8.0, will be removed in 0.9.0. Support for parsing catalog level identifier in Catalog identifiers is deprecated. Please refer to the table using only its namespace and its table name.
  _deprecation_warning(deprecat

In [20]:
dataset._table.last_sequence_number

3

Interestingly, the sequence number increase by 2 instead of 1. After some investigation, I couldn't find any documentation that explains this in details. But there are some hints in this docstring of `overwrite` method.

```md
        Shorthand for overwriting the table with a PyArrow table.

        An overwrite may produce zero or more snapshots based on the operation:

            - DELETE: In case existing Parquet files can be dropped completely.
            - REPLACE: In case existing Parquet files need to be rewritten.
            - APPEND: In case new data is being inserted into the table.
```

In this case, it seems to perform two operations `DELETE` and `APPEND`, thus two snapshots. We can inspect the snapshot history.

## Inspecting Snapshots

In [21]:
dataset._table.snapshots()


[
    Snapshot(
        snapshot_id=7841752524588546086,
        parent_snapshot_id=None,
        sequence_number=1,
        timestamp_ms=1732274297944,
        manifest_list='file://tmp/warehouse/default.db/taxi_dataset/metadata/snap-7841752524588546086-0-fecbe5f5-9fbd-4dca-9bcc-a775c57166a1.avro',
        summary=Summary(operation=Operation.APPEND),
        schema_id=0
    ),
    Snapshot(
        snapshot_id=651915520631835796,
        parent_snapshot_id=7841752524588546086,
        sequence_number=2,
        timestamp_ms=1732275203862,
        manifest_list='file://tmp/warehouse/default.db/taxi_dataset/metadata/snap-651915520631835796-0-2e76d850-273f-4910-a93c-90230bbae066.avro',
        summary=Summary(operation=Operation.DELETE),
        schema_id=0
    ),
    Snapshot(
        snapshot_id=5751889189087713943,
        parent_snapshot_id=651915520631835796,
        sequence_number=3,
        timestamp_ms=1732275204580,
        manifest_list='file://tmp/warehouse/default.db/taxi_d

You can see that there are consecutive snapshots like this. Now let's load the data to confirm this.
```
summary=Summary(operation=Operation.DELETE),
summary=Summary(operation=Operation.APPEND),
```

Let's grab the snapshot_id and load the data.

## Load Iceberg table with snapshot_id

In [30]:
_ = dataset._load(snapshot_id=651915520631835796)
_

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee


It looks like an empty table after `DELETE`, let's check the other snapshot.

In [34]:
_ = dataset._load(snapshot_id=dataset._table.snapshots()[-1].snapshot_id)

In [35]:
_.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## Use Tag/Branch for versioning


In [36]:
t = dataset._table

In [37]:
t.scan?

Signature:
t.scan(
    row_filter: 'Union[str, BooleanExpression]' = AlwaysTrue(),
    selected_fields: 'Tuple[str, ...]' = ('*',),
    case_sensitive: 'bool' = True,
    snapshot_id: 'Optional[int]' = None,
    options: 'Properties' = {},
    limit: 'Optional[int]' = None,
) -> 'DataScan'
Docstring:
Fetch a DataScan based on the table's current metadata.

    The data scan can be used to project the table's data
    that matches the provided row_filter onto the table's
    current schema.

Args:
    row_filter:
        A string or BooleanExpression that decsribes the
        desired rows
    selected_fileds:
        A tuple of strings representing the column names
        to return in the output dataframe.
    case_sensitive:
        If True column matching is case sensitive
    snapshot_id:
        Optional Snapshot ID to time travel to. If None,
        scans the table as of the current snapshot ID.
    options:
        Additional Table properties as a dictionary of
        string k

In [334]:
# Tags fits Kedro versioning better so I use tags instead of branch here
table = dataset._table

In [344]:
s = table.manage_snapshots()
(
    s.create_tag(
        snapshot_id=table.current_snapshot().snapshot_id, tag_name="2024-11-21"
    ).create_tag(snapshot_id=table.snapshots()[0].snapshot_id, tag_name="2024-11-20")
    .commit()
)
# Use the first snapshot as an "older" version

In [346]:
snapshot = table.snapshot_by_name("2024-11-21")
type(snapshot)

<class 'pyiceberg.table.snapshots.Snapshot'>

In [348]:
table.scan(snapshot_id=table.snapshot_by_name("2024-11-21")).to_arrow()

ValueError: Snapshot not found: Operation.APPEND: id=1188167320489258665, parent_id=6828333843454423456, schema_id=0

In [ ]:
UPDATE
col_a from orders where x=1

row_filter: = x=1
selected_fields = col_a from orders where x=1


In [343]:
s.create_tag??

Signature:
s.create_tag(
    snapshot_id: 'int',
    tag_name: 'str',
    max_ref_age_ms: 'Optional[int]' = None,
) -> 'ManageSnapshots'
Source:   
    def create_tag(self, snapshot_id: int, tag_name: str, max_ref_age_ms: Optional[int] = None) -> ManageSnapshots:
        """
        Create a new tag pointing to the given snapshot id.

        Args:
            snapshot_id (int): snapshot id of the existing snapshot to tag
            tag_name (str): name of the tag
            max_ref_age_ms (Optional[int]): max ref age in milliseconds

        Returns:
            This for method chaining
        """
        update, requirement = self._transaction._set_ref_snapshot(
            snapshot_id=snapshot_id,
            ref_name=tag_name,
            type="tag",
            max_ref_age_ms=max_ref_age_ms,
        )
        self._updates += update
        self._requirements += requirement
        return self
File:      ~/miniconda3/envs/kedro/lib/python3.10/site-packages/pyiceberg/table/__ini

In [338]:
table.metadata.refs


{
    'main': SnapshotRef(
        snapshot_id=1188167320489258665,
        snapshot_ref_type=SnapshotRefType.BRANCH,
        min_snapshots_to_keep=None,
        max_snapshot_age_ms=None,
        max_ref_age_ms=None
    )
}